In [1]:
# Python Script to handle basic AWS Operations by detecting Hand Gestures.

import boto3

myec2 = boto3.resource("ec2")
client = boto3.client("iam")
mybucket = boto3.resource("s3")

def launchOS():
    response = myec2.create_instances(
        ImageId="ami-05552d2dcf89c9b24",    #Varies for every instance type
        InstanceType = "t2.micro",
        MaxCount = 1,
        MinCount = 1
    )

def createUser():
    client.create_user(UserName = "AutoUser")

def createBucket():
    mybucket.create_bucket(Bucket = "autobucket1-sp",   #Must be a unique name
    CreateBucketConfiguration = {'LocationConstraint': 'ap-south-1'})

import  cv2
cap = cv2.VideoCapture( 0 )

from cvzone.HandTrackingModule import HandDetector
detector = HandDetector()

while True:
    status , photo = cap.read()
    cv2.imshow("my photo" , photo)
    if cv2.waitKey(100) == 13:
        break

    handphoto = detector.findHands(photo , draw=False)

    if handphoto:
        lmlist = handphoto[0]
        fingerstatus = detector.fingersUp(lmlist)

        if fingerstatus == [1, 1, 1, 1, 1]:
            print("All Fingers Up")
            launchOS()          #Launches EC2 Instance

        elif fingerstatus == [0, 1, 0, 0, 0]:
            print("Index Finger Up")
            createUser()        #Creates IAM User

        elif fingerstatus == [ 0, 1, 1, 0, 0 ]:
            print("Index and Middle Finger Up")
            createBucket()      #Creates S3 Bucket

        else:
            print("No Match Found!")


cv2.destroyAllWindows()


cap.release()